# **COVID 19 Impact on Ecommerce ASSIGNMENT**


---



Assignment

---



**Team:** Team Vishleshi


---











## **Abstract**

Foo bar..


---






## **Problem Statement & Business Context**

The details of the problem context and the analytical overview can be found HERE



---









## **Exploratory Data Analysis**

In [1]:
#!pip install gensim
#!pip install wordcloud

### Print Formats & Libraries

In [1]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [2]:
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot as plt
from scipy.stats import ttest_1samp, ttest_ind, mannwhitneyu, levene, shapiro
from statsmodels.stats.power import ttest_power
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm
from   statsmodels.formula.api import ols
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import zscore
from sklearn import svm
from platform import python_version
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora  as corpora

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
from wordcloud import WordCloud
import glob

print (color.BOLD + color.RED + "Solving for the current objective we will be using python version " + python_version())

Solving for the current objective we will be using python version 3.7.6


### Data Loading

In [3]:
data = pd.read_csv('all_ticketsnew.csv')
data.head()

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact,body-tokenized
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4,"['h', ' ', 'n', 'c', 'e', ' ', 'r', 'e', 'c', ..."
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4,"['c', 'n', ' ', 'e', 'r', ' ', 'p', 'l', 'e', ..."
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4,"['w', 'r', 'k', ' ', 'e', 'x', 'p', 'e', 'r', ..."
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4,"['r', 'e', 'q', 'u', 'e', 'n', 'g', ' ', 'e', ..."
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4,"['r', 'e', ' ', 'e', 'x', 'p', 'r', 'e', ' ', ..."


### Create Bigram & Trigram Modes Using Gensim

In [6]:
def sent_to_words (sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))
        
data_words=list(sent_to_words(data['body']))
data_words[:1]

[['hi',
  'since',
  'recruiter',
  'lead',
  'permission',
  'approve',
  'requisitions',
  'makes',
  'please',
  'make',
  'thanks',
  'recruiter']]

In [7]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print (trigram_mod[bigram_mod[data_words[0]]])

['hi', 'since', 'recruiter', 'lead', 'permission', 'approve', 'requisitions', 'makes', 'please', 'make', 'thanks', 'recruiter']


In [13]:
#Removing StopWords
nltk.download('stopwords')
stop_words=list(stopwords.words('english'))
# Removing Stop Words
stop_words=stop_words + ['hi','regards','thank','hello','sent','kind','help','dear','best','let']

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmitization(texts):
    accepted_tags=['NOUN','ADJ','VERB','ADV']
    texts_out=[]
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in accepted_tags])

[nltk_data] Downloading package stopwords to C:\Users\Aditya
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
data_words =  remove_stopwords(data_words)
data_words_bigrams =  make_bigrams(data_words)

print (data_words_bigrams[:1])

[['since', 'recruiter', 'lead', 'permission', 'approve', 'requisitions', 'makes', 'please', 'make', 'thanks', 'recruiter']]


In [15]:
id2word = corpora.Dictionary(data_words)
texts =  data_words
corpus = [id2word.doc2bow(text) for text in texts]

print (corpus[:1])
print ([[(id2word[id],freq) for id, freq in cp] for cp in corpus[:1]])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1)]]
[[('approve', 1), ('lead', 1), ('make', 1), ('makes', 1), ('permission', 1), ('please', 1), ('recruiter', 2), ('requisitions', 1), ('since', 1), ('thanks', 1)]]


## Baseline LDA Model

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

for topic in lda_model.print_topics():
    print (topic)
    print ('\n')

(0, '0.025*"sent" + 0.017*"message" + 0.013*"password" + 0.012*"error" + 0.010*"server" + 0.009*"information" + 0.008*"open" + 0.008*"contact" + 0.007*"using" + 0.007*"following"')


(1, '0.036*"sent" + 0.023*"approval" + 0.022*"please" + 0.020*"leave" + 0.020*"sa" + 0.020*"report" + 0.019*"submit" + 0.016*"days" + 0.016*"si" + 0.016*"manager"')


(2, '0.048*"please" + 0.038*"sent" + 0.024*"hi" + 0.021*"regards" + 0.019*"hello" + 0.018*"thank" + 0.016*"thanks" + 0.015*"access" + 0.015*"issue" + 0.012*"could"')




In [12]:
doc_lda = lda_model[corpus]
perpexity = lda_model.log_perplexity(corpus)
coherence_model = gensim.models.ldamodel.CoherenceModel(model=lda_model, 
                                texts=data_words,
                                dictionary=id2word,
                                coherence='c_v')

coherence_score = coherence_model.get_coherence()

print ('Perpexity score is %f and Coherence score is %f'%(perpexity,coherence_score))

Perpexity score is -7.140008 and Coherence score is 0.586244


In [20]:
data_words

[['since',
  'recruiter',
  'lead',
  'permission',
  'approve',
  'requisitions',
  'makes',
  'please',
  'make',
  'thanks',
  'recruiter'],
 ['icon',
  'please',
  'setup',
  'icon',
  'per',
  'icon',
  'engineers',
  'please',
  'details',
  'needed',
  'thanks',
  'lead'],
 ['work',
  'experience',
  'user',
  'work',
  'experience',
  'student',
  'coming',
  'next',
  'name',
  'much',
  'appreciate',
  'duration'],
 ['requesting',
  'meeting',
  'please',
  'follow',
  'equipments',
  'cable',
  'pc',
  'cord',
  'plug'],
 ['expire',
  'days',
  'ask',
  'update',
  'passwords',
  'colleagues',
  'pm',
  'expire',
  'days',
  'importance',
  'high',
  'received',
  'expiration',
  'please',
  'kindly',
  'prolongation',
  'pm',
  'expire',
  'days',
  'importance',
  'high',
  'expire',
  'days',
  'order',
  'change',
  'please',
  'follow',
  'steps',
  'prerequisites',
  'disable',
  'device',
  'credentials',
  'close',
  'active',
  'connected',
  'cable',
  'machine',
 

## Finding the dominant topic in each sentence 

In [18]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(data_words)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

TypeError: '<' not supported between instances of 'int' and 'tuple'

## Find the most representative document for each topic

In [19]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

#The tabular output above actually has 20 rows, one each for a topic. It has the topic number, 
#the keywords, and the most representative document. The Perc_Contribution column is nothing 
#but the percentage contribution of the topic in the given document.
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.9774,"internet, population, awareness, shopping, awa...","[comes, ratio, people, awareness, internet, sh..."
1,1.0,0.9762,"fear, card, net, businesses, platforms, many, ...","[onset, internet, wave, people, nowadays, turn..."
2,2.0,0.9740,"security, small, business, internet, product, ...","[giants, like, amazon, ebay, flipkart, better,..."
3,3.0,0.9828,"businesses, advertisements, computer, virus, m...","[computer, virus, formidable, execution, trans..."
4,4.0,0.9762,"delivery, per, business, low, internet, countr...","[internet, backbone, unfortunately, internet, ..."


## Topic distribution across documents

In [20]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,1.0,"fear, card, net, businesses, platforms, many, ...",39.0,0.2378
1.0,0.0,"internet, population, awareness, shopping, awa...",21.0,0.1280
2.0,3.0,"businesses, advertisements, computer, virus, m...",25.0,0.1524
3.0,2.0,"security, small, business, internet, product, ...",21.0,0.1280
4.0,0.0,"internet, population, awareness, shopping, awa...",23.0,0.1402
...,...,...,...,...
159.0,3.0,"businesses, advertisements, computer, virus, m...",NaN,NaN
160.0,1.0,"fear, card, net, businesses, platforms, many, ...",NaN,NaN
161.0,3.0,"businesses, advertisements, computer, virus, m...",NaN,NaN
162.0,4.0,"delivery, per, business, low, internet, countr...",NaN,NaN
